In [ ]:
#!pip install fastembed langchain langchain-community PyMuPDF chromadb pypdf

In [9]:
#!pip list

In [6]:
from langchain_community.llms import Ollama

#FastEmbed: Para generar embeddings rápidamente.
#LangChain: Para integrar modelos de lenguaje y gestionar flujos de trabajo.
#LangChain Community: Para integraciones adicionales.
#PyMuPDF: Para extraer texto de documentos PDF.
#Chroma: Para almacenar datos vectoriales y buscar embeddings.

llm_ms4m = Ollama(model="mistral")

llm_ms4m.invoke("Hola, quien eres")

C:\Users\BryanInche-MS4M\AppData\Local\Temp\ipykernel_30984\251033526.py:9: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm_ms4m = Ollama(model="mistral")


'¡Hola! Soy un asistente de inteligencia artificial creado para ayudarte en tus preguntas y necesidades. ¿Puedo hacer algo por ti hoy?'

## RAG

### Cargar los Documentos 

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader_documento = PyPDFLoader("ASIS_REPORTE_EJECUTIVO.pdf")

In [8]:
data_asis_ms4m = loader_documento.load()

In [11]:
data_asis_ms4m[1]

Document(metadata={'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2024-05-22T17:50:58-05:00', 'author': 'Fernando Mantari - MS4M', 'moddate': '2024-05-22T17:50:58-05:00', 'source': 'ASIS_REPORTE_EJECUTIVO.pdf', 'total_pages': 12, 'page': 1, 'page_label': '2'}, page_content='INFORME DE INTEGRACIÓN DE DATOS SMEB \n1. Integración de Datos \n \n1.1. Introducción General \n \nEl siguiente informe . Detalla la construcción de los 4 reportes ejecutivos que se \npresenta en la Sociedad Minera el Brocal. Desde e origen de las bases de datos y \nsu procedencia. \n1.2. Resumen \n \nEn SMEB se esta realizando 4 reportes ejecutivos que son entregados a la gerencia \ncada fin de turno a las 8am. Para ello, se ha creado un archivo Excel automatizado \npara cada uno de ellos, los cuales son alimentados por diferentes fuentes de \ndatos, provenientes del área de Despacho y Operaciones Mina.  Cada uno de los \nreportes tiene diferentes

### Chunks del Documento

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# chunk_size tamaño máximo de cada fragmento (chunk) de texto
# 512-2048 tokens (depende del modelo)

# chunk_overlap cuántos caracteres o tokens se superponen entre fragmentos consecutivos 
# 10%-20% del chunk_size

#  Memoria del LLMs 

texto_spliter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap = 500)

documentos = texto_spliter.split_documents(data_asis_ms4m)

In [21]:
documentos[1]

Document(metadata={'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2024-05-22T17:50:58-05:00', 'author': 'Fernando Mantari - MS4M', 'moddate': '2024-05-22T17:50:58-05:00', 'source': 'ASIS_REPORTE_EJECUTIVO.pdf', 'total_pages': 12, 'page': 1, 'page_label': '2'}, page_content='INFORME DE INTEGRACIÓN DE DATOS SMEB \n1. Integración de Datos \n \n1.1. Introducción General \n \nEl siguiente informe . Detalla la construcción de los 4 reportes ejecutivos que se \npresenta en la Sociedad Minera el Brocal. Desde e origen de las bases de datos y \nsu procedencia. \n1.2. Resumen \n \nEn SMEB se esta realizando 4 reportes ejecutivos que son entregados a la gerencia \ncada fin de turno a las 8am. Para ello, se ha creado un archivo Excel automatizado \npara cada uno de ellos, los cuales son alimentados por diferentes fuentes de \ndatos, provenientes del área de Despacho y Operaciones Mina.  Cada uno de los \nreportes tiene diferentes

In [22]:
documentos[2]

Document(metadata={'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2024-05-22T17:50:58-05:00', 'author': 'Fernando Mantari - MS4M', 'moddate': '2024-05-22T17:50:58-05:00', 'source': 'ASIS_REPORTE_EJECUTIVO.pdf', 'total_pages': 12, 'page': 2, 'page_label': '3'}, page_content='2. Esquema 1 – Reporte de Extracción, por Destino y Tratamiento Planta \n \nILUSTRACIÓN 1: FLUJOGRAMA DE EXTRACCIÓN DE DATOS - REPORTE DE EXTRACCIÓN, POR DESTINO Y TRATAMIENTO PLANTA')

### Embedings de los Chunks, para guardar en el VectorStore

Mistral no recibe vectores directamente. Vamos a aclarar el flujo paso a paso:
- 1. Pregunta del usuario:

El usuario ingresa una pregunta, por ejemplo: "¿Qué significa ms3m-solutions?"

Embedding de la pregunta:

- 2. La pregunta se convierte en un vector usando un modelo de embeddings (por ejemplo, FastEmbedEmbeddings).

Este vector podría ser algo como [1, 7, 4, 0, 0, 1] (solo un ejemplo simplificado).

- 3. Búsqueda en el vector store:

El vector de la pregunta se compara con los vectores de los documentos en el vector store (Chroma) para encontrar los chunks más relevantes.

Estos chunks son texto plano, no vectores.

- 4. Respuesta de Mistral:

Los chunks relevantes (texto plano) se pasan como contexto a Mistral.

Mistral no recibe vectores, sino texto plano. Por ejemplo:

- 5.Ejemplo
Contexto: "ms3m-solutions es una empresa especializada en soluciones tecnológicas."
Pregunta: "¿Qué significa ms3m-solutions?"
Mistral genera una respuesta basada en el contexto y la pregunta.

In [24]:
# Mistral no necesita entender los vectores. 
# Solo recibe texto plano (el contexto y la pregunta) y genera una respuesta
# basada en su entrenamiento previo.

# Los vectores solo se usan en la fase de recuperación 
# (para encontrar los chunks relevantes). Una vez que se recuperan los chunks, 
# se convierten en texto plano y se pasan a Mistral.

In [51]:
#!pip list

In [ ]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

# Modelo Embeding Open Source, multilingue 
# Búsqueda semántica , búsqueda por similitud de embeddings 
embeding_modelo = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\BryanInche-MS4M\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]c:\Users\BryanInche-MS4M\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\BryanInche-MS4M\AppData\Local\Temp\fastembed_cache\models--qdrant--all-MiniLM-L6-v2-onnx. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitatio

### Generamos el VectorStore (Base de datos vectoriales)

Bases de datos vectoriales gratuitas y open source: Chroma, FAISS, Weaviate, Milvus, Qdrant y Redis.

#### Crear y guardar el vector store por primera vez.

In [ ]:
from langchain_community.vectorstores import Chroma


# Tomar los textos de los chunks y llevalos a vectores

#Guarda la base de datos en el directorio chroma_db_ms4m con el nombre de colección 
# data_despacho_new_ms4m.
vectore_store = Chroma.from_documents(
    documents=documentos, 
    embedding=embeding_modelo, # Convierte los documentos (chunks de texto) en vectores usando el modelo de embeddings
    persist_directory="chroma_db_ms4m", # Almacena estos vectores en una base de datos vectorial
    collection_name="data_despacho_new_ms4m"    
)

### Construimos el Retrieval

#### Cargar el vector store existente para usarlo en tu aplicación.

In [ ]:
## 5. Cargar el vector store existente (en aplicaciones posteriores)
# El retrival, hara una busqueda de los vectores mas cercanos en el vector store,
# y nos devolvera los mejores contextos que entregara en texto plano al LLM

vs_ms4m = Chroma(embedding_function=embeding_modelo, persist_directory="chroma_db_ms4m",
                 collection_name="data_despacho_new_ms4m")


# Crear el retriever, para que la busqueda se use el vector store que hemos creado
retriver_ms4m = vs_ms4m.as_retriever(search_kwargs={"k":2})

C:\Users\BryanInche-MS4M\AppData\Local\Temp\ipykernel_30984\2358776816.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vs_ms4m = Chroma(embedding_function=embeding_modelo, persist_directory="chroma_db_ms4m",


#### Validacion del RAG , mediante un PROMPT

In [34]:
from langchain.prompts import PromptTemplate

prompt_template_prueba = """Usa la siguiente informacion para responder al usuario.
Si no sabes algo, no inventes una respuesta solo en esos casos menciona que no lo sabes.

Contexto : {context}
Pregunta : {question}

Solo devuelve la respuesta util y responde unicamente en el idioma español
Respuesta Util:
"""

# Le decimos cual es el prompt, y que deberia usar como inputs el contexto y la pregunta
prompt_1 = PromptTemplate(template=prompt_template_prueba,
                            input_variables=["contexto", "pregunta"])

#### Construimos una cadena de Retrival, para que consuma el retriver construido

In [35]:
from langchain.chains import RetrievalQA

# crear una cadena de RetrievalQA en LangChain 
cadena_retrival = RetrievalQA.from_chain_type(
    llm=llm_ms4m,                      # Modelo de lenguaje (por ejemplo, Mistral)
    chain_type="stuff",                # Tipo de cadena (en este caso, "stuff")
    retriever=retriver_ms4m,           # Retriever que busca los documentos relevantes
    return_source_documents=True,      # Devuelve los documentos fuente utilizados
    chain_type_kwargs={"prompt": prompt_1}  # Prompt personalizado
)

In [36]:
respuesta_1 = cadena_retrival.invoke({"query": "Que es la BD LeyCh?"})
respuesta_1

{'query': 'Que es la BD LeyCh?',
 'result': ' La Base de Datos (BD) LeyCH es una base de datos utilizada para almacenar información sobre los viajes de carga realizados por SMEB. Es alimentada a partir de un excel llamado "Blending" y los datos son extraídos manualmente mediante un grupo de WhatsApp llamado "Blending – MS4M". Los campos que se llenan en la base de datos incluyen la ley, los números de viajes M0, M1, M2, ECONOMICO y MIXTO. Los campos azul son formulas.',
 'source_documents': [Document(metadata={'author': 'Fernando Mantari - MS4M', 'creationdate': '2024-05-22T17:50:58-05:00', 'creator': 'Microsoft® Word para Microsoft 365', 'moddate': '2024-05-22T17:50:58-05:00', 'page': 7, 'page_label': '8', 'producer': 'Microsoft® Word para Microsoft 365', 'source': 'ASIS_REPORTE_EJECUTIVO.pdf', 'total_pages': 12}, page_content='5.3. BD LeyCH \nLa base de datos de BD LeyCh tiene 10 campos a ser llenados, de los cuales son \nextraídos de un excell llamado “Blending”. Este ultimo es envi

In [37]:
respuesta_1["result"]

' La Base de Datos (BD) LeyCH es una base de datos utilizada para almacenar información sobre los viajes de carga realizados por SMEB. Es alimentada a partir de un excel llamado "Blending" y los datos son extraídos manualmente mediante un grupo de WhatsApp llamado "Blending – MS4M". Los campos que se llenan en la base de datos incluyen la ley, los números de viajes M0, M1, M2, ECONOMICO y MIXTO. Los campos azul son formulas.'

In [45]:
respuesta_2 = cadena_retrival.invoke({"query": "El grupo de WhatsApp llamado “Blending MS4M” quien lo creo?"})
respuesta_2["result"]

' El grupo de WhatsApp llamado "Blending – MS4M" fue creado por los encargados de transporte SMEB.'

In [49]:
respuesta_3 = cadena_retrival.invoke({"query": "que es Stock Canchas"})
respuesta_3["result"]

' El archivo Stock Canchas es un registro de los materiales y recursos utilizados en las canchas, proporcionado por la área de topografía. Este archivo se utiliza para llenar la hoja de stock "Bancos - Libres" y para realizar cálculos como el forecast 4 + 8, entre otros.'

In [43]:
respuesta_2["source_documents"]

[Document(metadata={'author': 'Fernando Mantari - MS4M', 'creationdate': '2024-05-22T17:50:58-05:00', 'creator': 'Microsoft® Word para Microsoft 365', 'moddate': '2024-05-22T17:50:58-05:00', 'page': 0, 'page_label': '1', 'producer': 'Microsoft® Word para Microsoft 365', 'source': 'ASIS_REPORTE_EJECUTIVO.pdf', 'total_pages': 12}, page_content='[Fecha] \n \n  \nFernando Mantari - MS4M \n[NOMBRE DE LA EMPRESA] \nÁREA:  DESPACHO UG SMEB \nREPORTABILIDAD'),
 Document(metadata={'author': 'Fernando Mantari - MS4M', 'creationdate': '2024-05-22T17:50:58-05:00', 'creator': 'Microsoft® Word para Microsoft 365', 'moddate': '2024-05-22T17:50:58-05:00', 'page': 7, 'page_label': '8', 'producer': 'Microsoft® Word para Microsoft 365', 'source': 'ASIS_REPORTE_EJECUTIVO.pdf', 'total_pages': 12}, page_content='5.3. BD LeyCH \nLa base de datos de BD LeyCh tiene 10 campos a ser llenados, de los cuales son \nextraídos de un excell llamado “Blending”. Este ultimo es enviado por los encargados \nde transporte 

In [44]:
metadata = []

for _ in respuesta_2["source_documents"]:
    metadata.append((_.metadata["page"], _.metadata["source"]))
metadata

[(0, 'ASIS_REPORTE_EJECUTIVO.pdf'), (7, 'ASIS_REPORTE_EJECUTIVO.pdf')]

In [52]:
!pip list

Package                                  Version
---------------------------------------- -----------
absl-py                                  2.1.0
aiohappyeyeballs                         2.4.6
aiohttp                                  3.11.12
aiosignal                                1.3.2
annotated-types                          0.7.0
anyio                                    4.8.0
asgiref                                  3.8.1
asttokens                                2.4.1
astunparse                               1.6.3
attrs                                    25.1.0
backoff                                  2.2.1
bcrypt                                   4.2.1
build                                    1.2.2.post1
cachetools                               5.5.1
certifi                                  2024.7.4
charset-normalizer                       3.3.2
Chroma                                   0.2.0
chroma-hnswlib                           0.7.6
chromadb                                